# Sprawdzanie środowiska

In [ ]:
from platform import python_version

print(python_version())

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Przygotowanie odpowiednich danych

In [ ]:
'''
Dla PoC wykonuje obliczenia dla:
 * '../data/DogsCats'
Folder docelowy:
 * '../data/Photos'
'''

dir_path = '../data/Photos'
A_folder = 'originals'
B_folder = 'photoshops'

In [ ]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import math
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import random
import os

# fix random bo tak ( ͡° ͜ʖ ͡°)
odp = 42
numpy.random.seed(odp)

# Stałe

In [ ]:
# Wilkości

IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

# Przygotowanie Danych

In [ ]:
'''
Opis danych:
1 - klasa 1 -> Originals
0 - klasa 2 -> Photoshops
''' 

A_folder_list = os.listdir(dir_path + '/' + A_folder)
B_folder_list = os.listdir(dir_path + '/' + B_folder)

filenames = []
categories = []

for filename in A_folder_list:
    categories.append(0)
    filenames.append(dir_path + '/' + A_folder + '/' + filename)

for filename in B_folder_list:
    categories.append(1)
    filenames.append(dir_path + '/' + B_folder + '/' + filename)


df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
# Mieszamy!
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
sample = random.choice(df['filename'])
image = load_img(sample)
plt.imshow(image)

# Obróbka zdjęć

In [ ]:
from skimage.feature import hog
from skimage.color import rgb2gray

In [ ]:
np_image = numpy.array(image)
plt.imshow(np_image)
plt.title('Obraz źródłowy')
plt.show()

np_gray = rgb2gray(np_image)
plt.imshow(np_gray, cmap='gray')
plt.title('Obraz czarno-biały')
plt.show()

hog_features, hog_image = hog(np_gray,
                              visualize=True,
                              block_norm='L2-Hys',
                              pixels_per_cell=(32, 32))

plt.imshow(hog_image, cmap='gray')
plt.title('Histogram kierunkowych gradientów')
plt.show()

In [ ]:
import mahotas
import cv2

def ft_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hu_moments = cv2.HuMoments(cv2.moments(image)).flatten()
    return hu_moments

def ft_haralick(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(image).mean(axis=0)
    return haralick

def ft_histogram(image, mask=None):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # chanels: [0,1,2], bo mamy przestrzeń HSV
    # mask: None
    # histSize: [8, 8, 8], bin count, po 8 dla każdego z kanałów
    # ranges : [0, 256, 0, 256, 0, 256], wszystko dla każdego z 3 kanałów
    hist  = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def ft_hog(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features = hog(image, block_norm='L2-Hys', pixels_per_cell=(32, 32))
    return hog_features

def preprocess_image(image_path, verbose=False):
    image = cv2.imread(image_path)
    image = cv2.resize(image, IMAGE_SIZE)
    histogram = ft_histogram(image)
    haralick = ft_haralick(image)
    hu = ft_hu_moments(image)
    hog = ft_hog(image)
    global_feature = numpy.hstack([histogram, haralick, hu, hog])
    if verbose:
        print(f'Histagram shape: {histogram.shape}')
        print(f'harlick shape: {haralick.shape}')
        print(f'Hu shape: {hu.shape}')
        print(f'Hog shape{hog.shape}')
        print('')
    return global_feature

def preprocess_image_no_hog(image_path, verbose=False):
    image = cv2.imread(image_path)
    image = cv2.resize(image, IMAGE_SIZE)
    histogram = ft_histogram(image)
    haralick = ft_haralick(image)
    hu = ft_hu_moments(image)
    global_feature = numpy.hstack([histogram, haralick, hu])
    if verbose:
        print(f'Histagram shape: {histogram.shape}')
        print(f'harlick shape: {haralick.shape}')
        print(f'Hu shape: {hu.shape}')
        print('')
    return global_feature
    

# Przeliczenie Cech Zdjęć + Kategorii

In [ ]:
g_f = []
g_f_n_h = []
l = []

for i in df['filename']:
    data_img = preprocess_image(i)
    data_img_no_hog = preprocess_image_no_hog(i)
    g_f.append(data_img)
    g_f_n_h.append(data_img_no_hog)

for i in df['category']:
    if i == 0:
        l.append('original')
    else:
        l.append('photoshop')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

print(f'Wektor zdjęć z HOG: {numpy.array(g_f).shape}')
print(f'Wektor zdjęć bez HOG: {numpy.array(g_f_n_h).shape}')
print(f'Wektor kategorii słownych: {numpy.array(l).shape}\n')

# Label Encoder
# Koty - 0
# Psy - 1
targetNames = numpy.unique(l)
le = LabelEncoder()
target = le.fit_transform(l)

# Scaler
scaler = MinMaxScaler(feature_range=(0, 1))
rescaled_features_gf = scaler.fit_transform(g_f)
rescaled_features_gfnh = scaler.fit_transform(g_f_n_h)

print(f'Wektor kategorii liczbowych: {target.shape}')
print(f'Skalowany wektor zdjęć z HOG: {numpy.array(rescaled_features_gf).shape}')
print(f'Skalowany wektor zdjęć bez HOG: {numpy.array(rescaled_features_gfnh).shape}\n')

print(f'Max arg z nieskalowanego(z HOG): {numpy.argmax(g_f[0])}, Min arg z nieskalowanego(z HOG): {numpy.argmin(g_f[0])}')
print(f'Max z nieskalowanego(z HOG): {numpy.amax(g_f[0])}, Min z nieskalowanego(z HOG): {numpy.amin(g_f[0])}\n')

print(f'Max arg z nieskalowanego(bez HOG): {numpy.argmax(g_f_n_h[0])}, Min arg z nieskalowanego(bez HOG): {numpy.argmin(g_f_n_h[0])}')
print(f'Max z nieskalowanego(bez HOG): {numpy.amax(g_f_n_h[0])}, Min z nieskalowanego(bez HOG): {numpy.amin(g_f_n_h[0])}\n')

print(f'Max arg z skalowanego(z HOG): {numpy.argmax(rescaled_features_gf[0])}, Min arg z skalowanego(z HOG): {numpy.argmin(rescaled_features_gf[0])}')
print(f'Max z skalowanego(z HOG): {numpy.amax(rescaled_features_gf[0])}, Min z skalowanego(z HOG): {numpy.amin(rescaled_features_gf[0])}\n')

print(f'Max arg z skalowanego(bez HOG): {numpy.argmax(rescaled_features_gfnh[0])}, Min arg z skalowanego(bez HOG): {numpy.argmin(rescaled_features_gfnh[0])}')
print(f'Max z skalowanego(bez HOG): {numpy.amax(rescaled_features_gfnh[0])}, Min z skalowanego(bez HOG): {numpy.amin(rescaled_features_gfnh[0])}')

# Zapis/Odczyt

In [ ]:
import h5py

def save(features, labels, dataframe, name):
    h5f_data = h5py.File('data_' + str(name) + '.h5', 'w')
    h5f_data.create_dataset('dataset', data=numpy.array(features))

    h5f_label = h5py.File('labels_' + str(name) + '.h5', 'w')
    h5f_label.create_dataset('dataset', data=numpy.array(labels))

    h5f_data.close()
    h5f_label.close()

    dataframe.to_csv('dataframe_' + str(name) + '.csv')
    
def load(features, labels, dataframe):
    h5f_data  = h5py.File(features, 'r')
    h5f_label = h5py.File(labels, 'r')

    global_features_string = h5f_data['dataset']
    global_labels_string   = h5f_label['dataset']

    global_features = numpy.array(global_features_string)
    global_labels   = numpy.array(global_labels_string)

    h5f_data.close()
    h5f_label.close()
    
    df = pd.read_csv(dataframe, index_col = 0)  
    
    return global_features, global_labels, df
    
save(rescaled_features_gf, target, df, name='Photos_with_HOG')
save(rescaled_features_gfnh, target, df, name='Photos_without_HOG')

# B od BIG - więcej danych, S - small
global_features_B, global_labels_B, df_B = load('data_Photos_with_HOG.h5', 'labels_Photos_with_HOG.h5', 'dataframe_Photos_with_HOG.csv')
global_features_S, global_labels_S, df_S = load('data_Photos_without_HOG.h5', 'labels_Photos_without_HOG.h5', 'dataframe_Photos_without_HOG.csv')

In [ ]:
print(f'Z HOG:')
print(f'\t Wektor zdjęć: {global_features_B.shape}')
print(f'\t Wektor kategorii słownych: {global_labels_B.shape}\n')
print(f'\t Wektor dataframe: {df_B.shape}\n\n')

print(f'BEZ HOG:')
print(f'\t Wektor zdjęć: {global_features_S.shape}')
print(f'\t Wektor kategorii słownych: {global_labels_S.shape}\n')
print(f'\t Wektor dataframe: {df_S.shape}\n')

In [ ]:
df_B.head()

In [ ]:
df_S.head()

# Funkcję liczące statystyki

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.metrics import recall_score, f1_score

def countStats(_y_true, _y_pred):
    accuracy = accuracy_score(_y_true, _y_pred, normalize=True)
    precision = precision_score(_y_true, _y_pred, average='weighted')
    recall = recall_score(_y_true, _y_pred, average='weighted')
    fscore = f1_score(_y_true, _y_pred, average='weighted')
    
    return accuracy, precision, recall, fscore

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

def plot_cm(cm, classes):
    plot_confusion_matrix(conf_mat=cm,
                          colorbar=True, 
                          show_absolute=True,
                          show_normed=True,
                          class_names=classes)

# Funkcje do liczenia modelu

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA

def do_the_thing(features, labels, pca_val=False):
    
    tcm_list = []
    tAccuracy_list = []
    tPrecision_list = []
    tRecall_list = []
    tFScore_list = []
    
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=odp)
    
    if pca_val:
        pca = PCA(n_components=532)
        features = pca.fit_transform(features)
    
    for train_index, test_index in kf.split(global_features_S, global_labels_S):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        
        svm = SVC(kernel='linear', probability=True, random_state=odp, verbose=True)
        svm.fit(X_train, y_train)
        
        y_pred = svm.predict(X_test)
        accuracy, precision, recall, fscore = countStats(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)
        
        tAccuracy_list.append(accuracy)
        tPrecision_list.append(precision)
        tRecall_list.append(recall)
        tFScore_list.append(fscore)
        tcm_list.append(cm)

    
    print(f'Kształt danych:')
    print(f'\t X_train: {X_train.shape}')
    print(f'\t X_test: {X_test.shape}')
    print(f'\t y_train: {y_train.shape}')
    print(f'\t y_test: {y_test.shape}')
    
    accuracy = numpy.mean(tAccuracy_list)
    precision = numpy.mean(tPrecision_list)
    recall = numpy.mean(tRecall_list)
    fscore = numpy.mean(tFScore_list)
    
    print(f'\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF-score: {fscore}\n')
    
    cm = sum(tcm_list)
    
    plot_cm(cm, ['Originals', 'Photoshops'])
    
    print(f'\n{cm}\n')


# Bez HOG

In [ ]:
do_the_thing(global_features_S, global_labels_S, pca_val=False)

# Z HOG -> PCA

In [ ]:
do_the_thing(global_features_B, global_labels_B, pca_val=True)

# Dobór parametrów

In [ ]:
'''
TBD
from sklearn.model_selection import GridSearchCV


model = SVC()

param_grid = [
                {'kernel': ['rbf'], 'gamma': ['scale', 'auto'],'C': [1, 10, 100, 1000]},
                {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}                             
              ]


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=2, verbose=1)
grid_result = grid.fit(X_train, y_train)

print("Najlepsze acc: %f używając %s" % (grid_result.best_score_, grid_result.best_params_))

# Wyniki!
grid_result.best_params_
'''

print('TBD')

In [ ]:
'''
ToDo:
 - [ ] GridSearch
 - [ ] Sprawdzić output - predict
'''